In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Carichiamo ed Ispezioniamo i dati

In [12]:
df= pd.read_csv("../data/raw/BoxOfficeCollections.csv")

In [19]:
#prime 5 righe del dataset
df.head()

,Movie,Year,Score,Adjusted Score,Director,Cast,Consensus,Box Office Collection,Imdb_genre,IMDB Rating,metascore,time_minute,Votes
0,Hot Rod,2007,39,42.918,Akiva Schaffer,"Andy Samberg, Jorma Taccone, Bill Hader, Danny...","For Rod Kimball (Andy Samberg), performing stu...",14371564.0,Comedy,6.7,43.0,88.0,84956.0
1,Game Night,2018,85,99.838,John Francis Daley,"Jason Bateman, Rachel McAdams, Kyle Chandler, ...",Max and Annie's weekly game night gets kicked ...,117378084.0,Comedy,6.9,66.0,100.0,229292.0
2,The First Wives Club,1996,49,53.174,Hugh Wilson,"Goldie Hawn, Bette Midler, Diane Keaton, Maggi...",Despondent over the marriage of her ex-husband...,181489203.0,Comedy,6.4,58.0,103.0,48413.0
3,Scary Movie,2000,52,54.973,Keenen Ivory Wayans,"Shawn Wayans, Marlon Wayans, Cheri Oteri, Shan...","Defying the very notion of good taste, Scary M...",277200000.0,Comedy,6.2,48.0,88.0,254927.0
4,Blockers,2018,84,96.883,Kay Cannon,"Leslie Mann, Ike Barinholtz, John Cena, Kathry...","Julie, Kayla and Sam are three high school sen...",94523781.0,Comedy,6.2,69.0,102.0,78498.0


In [18]:
#ultime 5 righe del dataset
df.tail()

,Movie,Year,Score,Adjusted Score,Director,Cast,Consensus,Box Office Collection,Imdb_genre,IMDB Rating,metascore,time_minute,Votes
1373,Welcome to Chechnya,2020,100,104.537,David France,NaN,Activists risk their lives to confront Russian...,NaN,NaN,NaN,NaN,NaN,NaN
1374,White Riot,2019,100,102.793,Rubika Shah,"Pauline Black, Topper Headon, Mick Jones, Denn...",An exploration of how punk influenced politics...,185528.0,NaN,NaN,NaN,NaN,NaN
1375,The Woman Who Ran,2020,100,101.553,Hong Sang-soo,"Kim Min-hee, Seo Young-hwa, Sae-Byuk Kim, Kwon...","The 24th feature from Hong Sangsoo, THE WOMAN ...",NaN,NaN,NaN,NaN,NaN,NaN
1376,Woodlands Dark and Days Bewitched: A History o...,2021,100,101.978,Kier-La Janisse,"Piers Haggard, Lawrence Gordon Clark, Jeremy D...",WOODLANDS DARK AND DAYS BEWITCHED is the first...,NaN,NaN,NaN,NaN,NaN,NaN
1377,The Work,2017,100,103.657,Jairus McLeary,NaN,"Set inside a single room in Folsom Prison, thr...",NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
#dimensioni del dataset
df.shape

(1378, 13)

In [20]:
#info generali sul dataset
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 1378 entries, 0 to 1377
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Movie                  1378 non-null   str    
 1   Year                   1378 non-null   int64  
 2   Score                  1378 non-null   int64  
 3   Adjusted Score         1378 non-null   float64
 4   Director               1377 non-null   str    
 5   Cast                   1356 non-null   str    
 6   Consensus              1378 non-null   str    
 7   Box Office Collection  962 non-null    float64
 8   Imdb_genre             1024 non-null   str    
 9   IMDB Rating            1015 non-null   float64
 10  metascore              926 non-null    float64
 11  time_minute            1016 non-null   float64
 12  Votes                  1015 non-null   float64
dtypes: float64(6), int64(2), str(5)
memory usage: 140.1 KB


In [21]:
#conteggio dei valori nulli per ogni colonna del dataset
df.isnull().sum()

Movie                      0
Year                       0
Score                      0
Adjusted Score             0
Director                   1
Cast                      22
Consensus                  0
Box Office Collection    416
Imdb_genre               354
IMDB Rating              363
metascore                452
time_minute              362
Votes                    363
dtype: int64

In [23]:
#restituisce i duplicati del dataset se presenti
df.duplicated().sum()

np.int64(0)

In [26]:
#restituisce una descrizione generale del dataset
df.describe()
df.describe(include=["object"])

C:\Users\annac\AppData\Local\Temp\ipykernel_24648\4180965889.py:3: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  df.describe(include=["object"])


,Movie,Director,Cast,Consensus,Imdb_genre
count,1378,1377,1356,1378,1024
unique,1378,934,1345,1374,6
top,Hot Rod,Steven Spielberg,"Elijah Wood, Ian McKellen, Liv Tyler, Viggo Mo...",...,Comedy
freq,1,12,3,5,382
